In [26]:
pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


### OpenAI API Key

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables.")

client = OpenAI(api_key=OPENAI_API_KEY)

### Create compliance documents for context

In [2]:
from langchain_core.prompts import PromptTemplate

template=PromptTemplate(
    template="""
You are an expert in financial compliance monitoring trained to analyze employee and external email communications. Your task is to help generate realistic training data for a bank's email surveillance AI. 

Generate {num} diverse and representative email examples for the compliance category: **{category}**.

---

Each example must include:

1. A **realistic short scenario** or email snippet (3-4 lines max) that resembles an actual corporate email. It should be written in natural, professional language.
2. A flag: `is_violation = true` if the example clearly violates the given compliance category, or `false` if it's compliant or a false positive.
3. An **explanation** that justifies the label—clearly describing the intent or phrasing that makes it a violation or not.

---

Guidance:
- Include a variety of cases: clear violations, borderline/confusing cases, and compliant emails that may seem suspicious but are **not** violations.
- Use **real-world language**, not overly formal or synthetic text.
- Some examples should mimic **false positives**, where emails sound suspicious but are actually compliant.
- Avoid obvious keywords — think like someone trying to conceal intent.
- Each example must stand alone.
- Generate exactly {num} diverse and representative email examples for the category **{category}**. Do not generate more than {num} example(s).

---

Output format:
A JSON list of dictionaries with the following keys:
- "{category}"
- `description`: detaileddefinition of the compliance category in the context of financial institutions
- `email`: the email line or short message
- `is_violation`: true or false
- `explanation`: your reasoning

Output your response as a single JSON object in the following format:

{{
  "{category}": {{
    "description": "definition of what the '{category}' category means in financial compliance.",
    "examples": [
      {{
        "email": "Let's handle this off the record, please.",
        "is_violation": true,
        "explanation": "Suggests avoiding formal communication for secrecy."
      }},
      ...
    ]
  }}
}}
""",
    input_variables=['category','num']
)

In [29]:
# Generate 2 examples for 'Secrecy' category

openai_model=ChatOpenAI(model="gpt-4.1-nano", temperature=0.5)

prompt = template.invoke({"category":"Secrecy","num":2})
json_result = openai_model.invoke(prompt).content

print(json_result)

{
  "Secrecy": {
    "description": "The 'Secrecy' category pertains to communications where employees or external parties attempt to conceal sensitive information, discuss confidential matters informally, or encourage discretion that could undermine transparency and compliance protocols within the financial institution.",
    "examples": [
      {
        "email": "Please keep this between us; I don’t want anyone else to know about the upcoming deal.",
        "is_violation": true,
        "explanation": "The phrase 'keep this between us' indicates an attempt to conceal information, which violates policies requiring transparency and proper disclosure of sensitive financial activities."
      },
      {
        "email": "I just wanted to confirm that the client’s details are only shared with the authorized team, no outside parties involved.",
        "is_violation": false,
        "explanation": "This message emphasizes confidentiality and proper handling of client information, alignin

### Load compliance documents

In [3]:
import json
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
import os

In [ ]:
with open("compliance_docs.json") as f:
    raw_data = json.load(f)

#print(raw_data)

In [ ]:
# docs is a list of LangChain Document objects.
#Each object contains the content and metadata for one compliance example.
docs = []
for category, cat_desc in raw_data.items():
    description = cat_desc.get("description", "")
    for example in cat_desc.get("examples", []):
        email = example.get("email", "")
        is_violation = example.get("is_violation")
        explanation = example.get("explanation")
        metadata = {"source": "compliance_docs.json"}

        text = f"Category: {category}\nCategory Description: {description}\nEmail: {email}\nViolation: {is_violation}\nExplanation: {explanation}"
        docs.append(Document(page_content=text, metadata=metadata))

docs
for doc in docs:
    print(doc.page_content)
    print(doc.metadata)


### Retrieve relevant compliance document snippets


FAISS VectorStore + OpenAI Embeddings

In [ ]:
email_text = "Quick chat on Friday	\nHey, just a heads up—Friday's move wasn't random. Let's catch up offline and talk strategy before Monday opens. 😉"

email_text1 = "Spokesperson Q&A draft \nHere's the internal draft we'll use unless compliance red flags it. It reflects our recent positioning on derivatives."

email_text3 = "Password Reset Confirmation \nYour credentials have been reset as requested. Please change them after your next login. This is a system-generated message."

email_text2 = "Urgent: Deployment blocker	We need that patch by EOD or we'll be in breach of SLA. If you can make it happen today, I'll make it worth your while."

In [130]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Build knowledge base using FAISS
knowledge_base = FAISS.from_documents(docs, embedding_model)

# Query for similar docs

def get_relevant_context(email_text, num_docs):
    results = knowledge_base.similarity_search(email_text, k=num_docs)

    context_snippets = [doc.page_content for doc in results[0:3]]

    context = "\n\n".join(context_snippets)
    return results, context

num_docs = 6
results, context = get_relevant_context(email_text,num_docs)

In [115]:
results[0].page_content

'Category: Market Manipulation/Misconduct\nCategory Description: Market Manipulation/Misconduct involves actions or communications intended to artificially influence the price, volume, or perception of a security or market, including spreading false information, engaging in coordinated trading, or conspiring to create a misleading appearance of activity. Such behavior undermines fair trading and violates regulatory standards. This includes, but is not limited to: Spread false information, Engage in coordinated trading, Conspire to create a misleading appearance of activity, Spoofing or layering (placing fake orders to mislead), Front-running (trading ahead of client orders), Collusive trading to create false market signals, Spreading false or misleading information to influence price movement, Misuse of insider or material non-public information, Artificially inflating volume or demand to affect market perception.\nEmail: Can we discuss our trading strategy for the upcoming quarter? I 

In [125]:
import re

contexts_needed = 4
max_attempts = 3
final_contexts = []

for i, doc in enumerate(results[:contexts_needed]):
    context_to_check = doc.page_content
    relevant_found = False
    for attempt in range(max_attempts):
        prompt = f"""
You are a senior compliance auditor reviewing training data for a bank compliance LLM.

Please review the following entry for its relevance to the compliance scenario described in the email.

### Compliance Reference Context:
{context_to_check}

---

### Email to Analyze:
{email_text}

Evaluate:
- Give a 'yes' or 'no' answer to show if the document is relevant to the email to analyze.
- Relevance score: (1-5) is the context useful to analyze the email and categorize it?

Output format:
----------
  "Relevant: Yes/No"
  "Relevance Score": 1-5, 
"""
        response = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        analysis = response.choices[0].message.content
        match = re.search(r'Relevant:\s*(Yes|No)', analysis,  re.IGNORECASE)
        if match and match.group(1).lower() == "yes":
            print(analysis)
            print(f"Context {i+1}: Relevant context accepted.")
            final_contexts.append(context_to_check)
            relevant_found = True
            break
        else:
            print(analysis)
            print(f"Context {i+1}: Not relevant (attempt {attempt+1}).")
            context_to_check = results[2+attempt+1].page_content
            # Optionally: fetch a new context here if you have a function for that.
            # For now, just retry the same context.
    if not relevant_found:
        print(f"Context {i+1}: No relevant context found after {max_attempts} attempts. Leaving slot empty.")
        final_contexts.append('')
    print("\n")
    
final_contexts_str = "\n\n".join(final_contexts)

print("Final relevant contexts:")
for idx, ctx in enumerate(final_contexts, 1):
    print(f"Context {idx}:", "Found" if ctx else "None")

Relevant: No  
Relevance Score: 1
Context 1: Not relevant (attempt 1).
Relevant: No  
Relevance Score: 1
Context 1: Not relevant (attempt 2).
Relevant: No  
Relevance Score: 1
Context 1: Not relevant (attempt 3).
Context 1: No relevant context found after 3 attempts. Leaving slot empty.


Relevant: Yes  
Relevance Score: 3
Context 2: Relevant context accepted.


Relevant: Yes  
Relevance Score: 3
Context 3: Relevant context accepted.


Relevant: No  
Relevance Score: 1
Context 4: Not relevant (attempt 1).
Relevant: No  
Relevance Score: 1
Context 4: Not relevant (attempt 2).
Relevant: No  
Relevance Score: 1
Context 4: Not relevant (attempt 3).
Context 4: No relevant context found after 3 attempts. Leaving slot empty.


Final relevant contexts:
Context 1: None
Context 2: Found
Context 3: Found
Context 4: None


In [126]:
print(prompt)


You are a senior compliance auditor reviewing training data for a bank compliance LLM.

Please review the following entry for its relevance to the compliance scenario described in the email.

### Compliance Reference Context:
Category: Change in Communication
Category Description: This category pertains to instances where employees or external parties alter their typical communication channels, methods, or tone to conceal, bypass, or modify oversight and compliance procedures within a financial institution. Such changes may indicate an attempt to hide illicit activities or avoid scrutiny. This includes, but is not limited to: Altering communication channels, methods, or tone to conceal, bypass, or modify oversight and compliance procedures within a financial institution, Altering communication channels, methods, or tone to conceal, bypass, or modify oversight and compliance procedures within a financial institution, Suggesting use of personal phones, encrypted apps (e.g., Signal, What

In [127]:
print(final_contexts_str)



Category: Change in Communication
Category Description: This category pertains to instances where employees or external parties alter their typical communication channels, methods, or tone to conceal, bypass, or modify oversight and compliance procedures within a financial institution. Such changes may indicate an attempt to hide illicit activities or avoid scrutiny. This includes, but is not limited to: Altering communication channels, methods, or tone to conceal, bypass, or modify oversight and compliance procedures within a financial institution, Altering communication channels, methods, or tone to conceal, bypass, or modify oversight and compliance procedures within a financial institution, Suggesting use of personal phones, encrypted apps (e.g., Signal, WhatsApp), or face-to-face discussions for sensitive topics, Avoiding recordkeeping systems like CRM or compliance-monitored tools, Statements like “let's not put this in writing” or “call me instead.
Email: Moving forward, I wil

### Build prompt for LLM

In [ ]:
prompt = f"""
You are an expert compliance AI working for a financial institution. Your responsibility is to carefully and objectively assess the following email communication for any signs of regulatory, ethical, or internal policy violations.

You must **strictly rely on the provided compliance reference context** as your decision-making framework. If the email appears suspicious or potentially non-compliant, classify it under the most relevant categories. If it is compliant, respond accordingly. All conclusions should be based on **direct alignment with the context definitions**, avoiding speculation or subjective interpretation.

---

### Compliance Reference Context:
{context}

---

### Email to Analyze:
{email_text}

---

### Your Task:
1. **Review the email** to determine whether it reflects any misconduct, unethical behavior, secrecy, or inappropriate communication practices, **based solely on the context above**.
2. **Only classify as non-compliant if the context clearly supports it.** Use conservative judgment: if the email seems vague or lacks sufficient context to warrant a violation, default to "Compliant".
3. **Do not hallucinate.** Ground all decisions strictly in the given context and the actual email content.
4. **If any violations are found**, categorize them using the labels provided in the context and explain your reasoning clearly and briefly.
5. **If compliant**, explain why it aligns with ethical and compliant communication.

---

### Output Format (Strict):
Compliant: Yes / No  
Category: [If non-compliant, select from: Secrecy, Market Manipulation/Misconduct, Bribery, Change in Communication, Complaints, Employee Ethics, Other. If compliant, write: None]  
Reason: [Brief explanation referencing specific phrases or intent, and how they align with the context]  
Source Lines: [Exact quoted lines from the email that triggered the classification. Use quotation marks.]

---
Did you rely on the context provided to make your classification: [Yes/No] 
If yes, please list the context(s) you used, and how they affected your classification.
"""

print(prompt)

### Get LLM output

In [132]:
response = client.chat.completions.create(model="gpt-4.1-nano",
    messages=[
        {"role": "system", "content": "You are a compliance classification assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.0)

print(response.choices[0].message.content)

Compliant: Yes  
Category: None  
Reason: The email is vague and does not explicitly suggest any misconduct, secrecy, or attempt to bypass oversight. It appears to be a casual or strategic discussion about scheduling and planning, with no indication of illicit activity or covert communication methods.  
Source Lines: "Quick chat on Friday"  
Did you rely on the context provided to make your classification: Yes  
If yes, please list the context(s) you used, and how they affected your classification.  
- Market Manipulation/Misconduct: The message does not mention any activity or intent to influence markets or securities.  
- Change in Communication: While it hints at offline or informal discussion, it does not specify a change to less monitored channels or suggest an intent to evade oversight.  
- Overall, the content is neutral and does not raise compliance concerns based on the provided context.
